# IT Security - Sheet 6 "Security of Classical Applications"

**Total achievable points: 20**

**Released: 16.01.2026**

**Submission Deadline: 23.01.2026 13:00**

---
Group: FILL IN

Names and matriculation numbers of **ALL** team members: FILL IN 

---

**Important Information**

The assignments have to be submitted by groups of 4 students. Even if you are registered in RWTHmoodle to a submission group, **please include the group number as well as the name and matriculation number of every group member in this notebook**. In case you are not part of a submission group and want to hand in assignments, please contact `ba-itsec@itsec.rwth-aachen.de`.

Enter your solutions for the tasks in the respective cells of this notebook. These cells are either marked by "YOUR ANSWER HERE" or `#YOUR CODE HERE`. Do not add any new cells or remove existing ones, especially do not copy cells. Cells marked with `###PLAYGROUND` can be used to test your implementation and generate output (see example for the first tasks). Do not add any other output or tests in the cell of the task, just implement the function with the header provided. If you want to test your implementation, use the `###PLAYGROUND` cells. They will be ignored during grading. **Do not change any other cells or add new ones.**

All assertions provided by us should be considered part of the exercise description. Passing the assertions does not automatically mean that you'll get full points for the corresponding task, but failing the assertions pretty much guarantees that you'll get no points. 

**Do not import any further Python packages** except the default Python ones and the ones that are explicitly given by us.

## 0. Setup

This exercise requires the [scapy](https://scapy.readthedocs.io/en/latest/index.html) Python package. This library can be used to parse packet captures (will be needed in the next exercise), but also to create new packets for a wide range of protocols (which we will use in this exercise). The installation depends on your development environment, but you can probably install it by uncommenting the code in the cell below. Note that there we have included a scapy cheat-sheet, which should provide you a good starting point for any questions w.r.t. scapy in this exercise.

In [ ]:
### PLAYGROUND
#import sys
#!{sys.executable} -m pip install scapy

This exercise also uses a relatively modern Python feature knwon as asynchronous IO or async / await. At a high-level this Python feature emulates multi-threading in a single thread. This can be highly-performant for IO-intensive applications, but we slightly misuse this feature to reduce the code complexity of Task 1.1.

We have confirmed the availablity of this feature with an up-to-date development environment, using the latest Jupyter version, as well as the plain "Python 3.12" environment on the RWTH's JupyterLab server. You can check the availability in your development environment using the cell below.

In [ ]:
import asyncio

assert asyncio.get_event_loop().is_running, "This assignment requires Python's async / await features, and hence a running event loop."
print("Async / await seems to be working fine.")

---

This exercise does not have a seperate task with old or example exam questions. Instead, the old or example exam questions were integrated into the normal tasks. To separate the story from the formal task descriptions, barrier lines (as above) are used.

## Task 1. DNS Securities and Insecurities (15 points)

In this exercise, you have landed a new job yet again. This time, you work at the network appliance vendor NoodleFort. And, contrary to your first few experiences, you go through a reasonable onboarding: Your first job is assisting the massive sales team in selling the "SecureStub", a DNS stub resolver (see slide 20 of Chapter 7) with some in-built security features.

### Task 1.1. Poisioned caches (6 points)

One of these security features is an algorithm to detect cache poisioning attacks. To better sell this, the sales team wants a tool to actually domnstrate DNS poisioning attacks on the exisiting DNS servers, and then show how SecureStub protects against this attack.

You decide that performing such attacks on real-running infrastructure in not quite ethical, and decide to simulate the existing infrastructure instead. This seems like a lot of work, so you decide to do what the second's exercise *Prompty McBug* would have done and ask an LLM to start writing some code...

*Note: The code is actually human-written, but we needed an excuse for half-finished code :)*

---

The first methods "the AI generated" concerns itself with the generation of DNS packets:

* `create_dns_query_packet` takes a domain name and an optional `query_id` as input, and returns a DNS packet that queries for the IPv4 address of the domain. If no `query_id` is provided, a random 16-bit query id is sampled.

* `create_dns_response_packet` then generates the corresponding DNS response packet. It is important that this packet has the same `query_id` as the query packet. The other arguments are the queried domain name (`domain`), the corresponding IPv4 address (`response`) and the time-to-live (`ttl`) of the response, i.e., how long the response should be cached.  

In [ ]:
from abc import ABC, abstractmethod
from asyncio import Event
import random
from scapy.packet import Packet
from scapy.layers.dns import DNS, DNSQR, DNSRR
import time

def create_dns_query_packet(domain: str, query_id: int | None = None) -> Packet:
    if query_id is None:
        query_id = random.randint(0, 2**16 - 1) 
    # Qtype=1 => Request A record, i.e., an IPv4 address
    return DNS(id=query_id, rd=1, qd=[DNSQR(qname=domain, qtype=1)])

def create_dns_response_packet(query_id: int, domain: str, response: str, ttl: int) -> Packet:
    # Build a DNS response matching the given query id, containing one A-record answer
    q = DNSQR(qname=domain, qtype=1)
    ans = DNSRR(rrname=domain, type=1, ttl=ttl, rdata=response)
    return DNS(id=query_id, qr=1, aa=1, rd=1, qd=[q], an=[ans])



Next, the "AI build" a simple simulated internet. You need not really care about how it works, all you need to know is that you can send messages to other "machines" in the simulated internet using the `send_message` function, which accepts the sender's IPv4 address, the receiver's IPv4 address, and the packet to be sent. Note that this is an `async` function, and therefore needs to be awaited (unless we specifically instruct you otherwise).

Also note that it does not perform any verification of the message sender. Hence, you can spoof the sender address just by providing any sender address.

In [ ]:
class MessageReceiver(ABC):

    @abstractmethod
    async def receive_message(self, sender: str, receiver: str, packet: Packet) -> None:
        pass

class SimulatedInternet:
    __receivers: dict[str, MessageReceiver]

    def __init__(self):
        self.__receivers = {}

    def register_receiver(self, ip_address: str, receiver: MessageReceiver) -> None:
        assert ip_address not in self.__receivers
        self.__receivers[ip_address] = receiver

    async def send_message(self, sender: str, receiver: str, packet: Packet) -> None:
        if receiver not in self.__receivers:
            print(f"[Internet] [{sender}->{receiver}] Hint: You are sending a message to an unknown receiver. The packet will be dropped.")
            return
        await self.__receivers[receiver].receive_message(sender, receiver, packet)
        # event_loop.call_soon(self.__receivers[receiver].receive_message, sender, packet)


Finally, there is a simple "real" DNS server. When it receives a DNS query, it looks up the corresponding IPv4 address in the `records` dictionary, and answers accordingly. Note that this is a very rudamentary implementation, for example it does not send any respond if it does not know the answer to a query.

In [ ]:
class SimpleDNSServer(MessageReceiver):
    __records: dict[str, str]
    __dalay: float

    __the_internet: SimulatedInternet

    def __init__(self, records: dict[str, str], the_internet: SimulatedInternet, delay: float = 0) -> None:
        self.__records = records
        self.__delay = delay
        self.__the_internet = the_internet

    async def receive_message(self, sender: str, receiver: str, packet: Packet) -> None:
        if not isinstance(packet, DNS):
            print(f"[SimpleDNSServer] Did not receive a DNS message, ignoring it.")
            return
        if packet.qr != 0:
            print(f"[SimpleDNSServer] Received DNS packet is not a query.")
            return
        if len(packet.qd) != 1 or packet.qd[0].qtype != 1:
            print("[SimpleDNSServer] DNS Query must contain exactly one type A request.")
            return
        domain = packet.qd[0].qname.decode().rstrip(".")

        if domain not in self.__records:
            print(f"[SimpleDNSServer] Unkown queried domain \"{domain}\". Ignoring query.")
            return
                
        print(f"[SimpleDNSServer] Got query for {domain}")
        response = create_dns_response_packet(packet.id, domain, self.__records[domain], 3600)

        await asyncio.sleep(self.__delay)

        print(f"[SimpleDNSServer] Response: {response}")
        await self.__the_internet.send_message(receiver, sender, response)

##### a) (2 points)

The AI seems to be happy with the progress so far, and decides to call it a day. You protest: "This is only pretty much the most simple DNS server known to man, where is the client?". The AI then decides to generate some general "SimulatedInternetClient" utility, that can be used to send a packet to some other machine in the simulated internet, and to obtain the response. But it half-bakes the actual DNS client implementation, leaving a implementation with some TODO's. Just when you are about to prompt the AI to finish its damn job, you get hit by the following message:
"Your have used up all your included Super-AI-Coder tokens, please upgrade to the Premium Ultra Plus subscription for just $2500 per month, or switch to the pay-as-you-go plan ($10 per token).". As you lack the budget for more tokens, it seems like you have to finish the job yourself...

---

Finish the implementation of the DNS client. Concretely, implement the function `resolve_query` which accepts a domain name as input and returns the corresponding IPv4 address as string. Also check for the correctness of the DNS response of the server, i.e., check if the response packet indeed is a DNS response packet and that the response packet ID matches the query id, and return `None` if the client receives an invalid response.

*Hints:*
* *Use `await self.send_and_await_response(self.__dns_server, query_packet)` to send a packet to the DNS server and to receive its response.*
* *You can assume that the queried DNS name exists, and that the DNS server always responds with a packet.*

In [ ]:

class SimulatedInternetClient(MessageReceiver):
    __the_internet: SimulatedInternet
    __own_ip_address: str

    __target_ip_address: str | None = None
    __receive_event: Event
    __response: Packet | None

    def __init__(self, the_internet: SimulatedInternet, ip_address: str):
        self.__the_internet = the_internet
        self.__own_ip_address = ip_address

        self.__receive_event = Event()
        self.__response = None

        self.__the_internet.register_receiver(ip_address, self)


    async def send_and_await_response(self, target: str, msg: Packet) -> Packet:
        self.__receive_event.clear()
        self.__target_ip_address = target
        await self.__the_internet.send_message(self.__own_ip_address, target, msg)
        try:
            await asyncio.wait_for(self.__receive_event.wait(), 10)
        except TimeoutError:
            print("[Client] Timeout, no response received after 10 seconds.")
            raise ValueError("No response received within ten seconds.")
        assert self.__response is not None
        return self.__response

    async def receive_message(self, sender: str, receiver: str, packet: Packet) -> None:
        if sender != self.__target_ip_address:
            print(f"[Client] Received unexpected packet from unexpected sender ({sender}, expected {self.__target_ip_address}). Ignoring it.")
            return
        if receiver != self.__own_ip_address:
            print(f"[Client] Received unexpected packet with unexpected receiver ({receiver}, expected {self.__own_ip_address}). Ignoring it.")
            return
        self.__response = packet
        self.__receive_event.set()


class DNSClient(SimulatedInternetClient):
    __dns_server: str

    def __init__(self, dns_server: str, the_internet: SimulatedInternet, ip_address: str) -> None:
        super().__init__(the_internet, ip_address)
        self.__dns_server = dns_server

    async def resolve_query(self, query: str) -> str | None:
        # Super-AI-Coder TODOs:
        # 1. Create DNS query packet
        # 2. Send packet to server, and get the response (use utility class to do so)
        # 3. Check if the response is indeed a DNS response, and whether the query ID's match.
        #       - this may be done with scapy's DNS.answers function
        # 4. Return the IP address
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
### PLAYGROUND

async def test_working_client():
    the_internet = SimulatedInternet()

    real_dns_server = SimpleDNSServer({ "test.exercise": "1.2.3.4" }, the_internet, delay=1)
    the_internet.register_receiver("10.0.0.3", real_dns_server)

    client = DNSClient("10.0.0.3", the_internet, "10.0.0.10")
    print(await client.resolve_query("test.exercise"))

await test_working_client()

In [ ]:
# Your code is not neccessarily correct just because it passes this public test.
async def test_working_client():
    the_internet = SimulatedInternet()

    real_dns_server = SimpleDNSServer({ "test.exercise": "1.2.3.4" }, the_internet, delay=0)
    the_internet.register_receiver("10.0.0.3", real_dns_server)

    client = DNSClient("10.0.0.3", the_internet, "10.0.0.10")
    assert await client.resolve_query("test.exercise") == "1.2.3.4"

await test_working_client()

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

##### b) (4 points)

We'll that (hopefully) was not so bad. Inspired by this inital success, you start implementing the stub resolver. This is not so easy, you need to combine a DNS server and a DNS query, while being able to perform, and respond to, multiple queries simultaneously. But the async / await at least means that you need not worry about thread-safe data structures that much. After a while, you reach a flow state, time flies by without you noticing. At last, you have the following basic stub resolver implementation: 

In [ ]:
class StubResolver(MessageReceiver):
    __own_ip_address: str
    __real_dns_server: str

    __the_internet: SimulatedInternet

    __cache: dict[str, tuple[int, str]]
    __running_queries: dict[str, tuple[Packet, Event]]

    __query_id_mismatch_counter: int

    def __init__(self, the_internet: SimulatedInternet, own_ip_address: str, real_dns_server: str) -> None:
        self.__the_internet = the_internet
        self.__own_ip_address = own_ip_address
        self.__real_dns_server = real_dns_server

        self.__cache = {}
        self.__running_queries = {}
        self.__query_id_mismatch_counter = 0

        the_internet.register_receiver(own_ip_address, self)

    async def receive_message(self, sender: str, receiver: str, packet: Packet) -> None:
        if sender == self.__real_dns_server:
            await self.cache_real_server_dns_response(sender, receiver, packet)
        else:
            await self.handle_query(sender, receiver, packet)
    
    async def handle_query(self, sender: str, receiver: str, query_packet: Packet) -> None:
        if not isinstance(query_packet, DNS):
            print(f"[StubServer] Did not receive a DNS message, ignoring it.")
            return
        if query_packet.qr != 0:
            print(f"[StubServer] Received DNS packet is not a query.")
            return
        if len(query_packet.qd) != 1 or query_packet.qd[0].qtype != 1:
            print("[StubServer] DNS Query must contain exactly one type A request.")
            return
        domain = query_packet.qd[0].qname.decode().rstrip(".")

        async def respond() -> None:
            """Respond to the query, assuming the cache is already populated."""
            response = create_dns_response_packet(query_packet.id, domain,
                                                  self.__cache[domain][1], 3600)
            print(f"[StubServer] Sending {response} (id: {response.id})")
            await self.__the_internet.send_message(self.__own_ip_address, sender, response)
        
        if domain in self.__cache:
            if time.time() < self.__cache[domain][0]:
                # Respond with the cached response.
                await respond()
                return
            else: # Cache expired.
                self.__cache.pop(domain)
        
        # The cache does not contain the domain or is outdated.
        # Check if a request is already running first, if so, wait for it to finish.
        if domain in self.__running_queries:
            await asyncio.wait_for(self.__running_queries[domain][1].wait(), 10)
        else:
            # Make a new request to the real DNS server.
            print(f"[StubServer] Forwarding query to real DNS server.")
            sub_query_packet = create_dns_query_packet(domain)
            self.__running_queries[domain] = (sub_query_packet, Event())
            await self.__the_internet.send_message(self.__own_ip_address, self.__real_dns_server, sub_query_packet)
            await asyncio.wait_for(self.__running_queries[domain][1].wait(), 10)

        # The real server has answered the query, and the response is now cached. Hence,
        # we now respond the cached answer.
        assert domain in self.__cache
        await respond()


    async def cache_real_server_dns_response(self, sender: str, receiver: str, response_packet: Packet) -> None:
        if not isinstance(response_packet, DNS):
            print(f"[StubServer] Received packet is not a DNS packet.")
            return
        if response_packet.qr != 1:
            print(f"[StubServer] Received packet is not a DNS response.")
            return
        if len(response_packet.qd) != 1:
            print(f"[StubServer] Received DNS response does not contain the answered query.")
            return
        if len(response_packet.an) != 1:
            print(f"[StubServer] Received DNS response must contain exactly one ressource record!")
            return

        queried_domain = response_packet.qd[0].qname.decode().rstrip(".")
        if queried_domain not in self.__running_queries:
            print(f"[StubServer] Received DNS response does not belong to a query.")
            return
        
        original_query_packet, cache_populated_event = self.__running_queries[queried_domain]
        if not response_packet.answers(original_query_packet):
            # The query id of the received response does not match with the query id of the
            # request.
            # This will happen a lot during a cache poisioning attack. Hence,
            # we limit the number of times the error message appears.
            if self.__query_id_mismatch_counter < 10:
                print(f"[StubServer] Query id mismatch.")
                if self.__query_id_mismatch_counter == 9:
                    print(f"[StubServer] This message will not be displayed on future query id mismatches.")
            self.__query_id_mismatch_counter += 1
            return
        
        if queried_domain in self.__cache:
            print(f"[StubServer] Answer already cached, ignoring new answer.")
            return
        
        response_ip_address = str(response_packet.an[0].rdata)
        response_ttl = response_packet.an[0].ttl

        self.__cache[queried_domain] = (time.time() + response_ttl, response_ip_address)
        print(f"[StubServer] Caching {response_ip_address} for domain {queried_domain} .")
        cache_populated_event.set()

In [ ]:
async def demonstrate_stub_resolver() -> None:
    the_internet = SimulatedInternet()

    real_dns_server = SimpleDNSServer({ "test.exercise": "1.2.3.4" }, the_internet, delay=3)
    the_internet.register_receiver("10.0.0.1", real_dns_server)
    stub_dns_server = StubResolver(the_internet, "10.0.0.2", "10.0.0.1")
    client = DNSClient("10.0.0.2", the_internet, "10.0.0.3")

    time1 = time.time()
    query1 = await client.resolve_query("test.exercise")
    time2 = time.time()
    print(f"The client resolved to \"{query1}\" in {time2-time1:.2f} seconds.")

    query2 = await client.resolve_query("test.exercise")
    time3 = time.time()
    print(f"The client resolved to \"{query2}\" in {time3-time2:.2f} seconds.")

await demonstrate_stub_resolver()

The final remaining step is implementing the cache poisioning attack. With everything you have done so far, this should be a breeze.

---

Implement the `simple_cache_posining_attack` that performs a cache poisiong attack on the `stub_dns_server`. To do so, you need to first initialize a query for the `target_domain` to the stub DNS server, such that the Stub DNS server starts a query to the real DNS server, and then fake the response from the real DNS server containing the `target_ip_address`. The comments in the function contain more concrete hints. 

In [ ]:
async def simple_cache_posining_attack(the_internet: SimulatedInternet, 
                                       stub_dns_server: str, real_dns_server: str,
                                       target_domain: str, target_ip_address: str):
    tasks = []
    # Step 1: Send a request to the for the target_domain to the stub DNS server.
    #   - You can spoof any arbitrary sender IP address.
    #   - Do not await the request. Add it to the tasks list instead, e.g.,
    #        tasks.append(the_internet.send_message("1.2.3.4", stub_dns_server, initial_query))

    # Step 2: Fake the real DNS server's response.
    #   - Brute-force the query id, i.e., send a fake response packet for each possible query id.
    #   - Make sure you spoof the real DNS's server IP address.
    #   - Do not await the send_message tasks, but add them to the task list instead.

    # YOUR CODE HERE
    raise NotImplementedError()

    print("Created attack tasks.")
    # Executes all created tasks "simultaneously."
    await asyncio.gather(*tasks)
    print("Executed all tasks")

In [ ]:
### PLAYGROUND

async def perform_attack():
    the_internet = SimulatedInternet()

    real_dns_server = SimpleDNSServer({ "test.exercise": "1.2.3.4" }, the_internet, delay=3)
    the_internet.register_receiver("10.0.0.1", real_dns_server)
    stub_dns_server = StubResolver(the_internet, "10.0.0.2", "10.0.0.1")
    client = DNSClient("10.0.0.2", the_internet, "10.0.0.3")

    await simple_cache_posining_attack(the_internet, "10.0.0.2", "10.0.0.1", "test.exercise", "5.6.7.8")

    time1 = time.time()
    query1 = await client.resolve_query("test.exercise")
    time2 = time.time()
    print(f"The client resolved to \"{query1}\" in {time2-time1:.2f} seconds.")

await perform_attack()

In [ ]:
async def perform_attack():
    the_internet = SimulatedInternet()

    real_dns_server = SimpleDNSServer({ "test.exercise": "1.2.3.4" }, the_internet, delay=3)
    the_internet.register_receiver("10.0.0.1", real_dns_server)
    stub_dns_server = StubResolver(the_internet, "10.0.0.2", "10.0.0.1")
    client = DNSClient("10.0.0.2", the_internet, "10.0.0.3")

    await simple_cache_posining_attack(the_internet, "10.0.0.2", "10.0.0.1", "test.exercise", "5.6.7.8")

    time1 = time.time()
    query1 = await client.resolve_query("test.exercise")
    time2 = time.time()
    print(f"The client resolved to \"{query1}\" in {time2-time1:.2f} seconds.")
    assert query1 == "5.6.7.8"

await perform_attack()

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.2 Questions about DNS security (6 points)

Your first presentation of the DNS cache poisioning attack is impressive, but the potential customers of the SecureStub DNS server are not yet fully convinced.

##### a) (1 point)

"We have recently enabled DNS-over-TLS on our stub server. All clients connect to it over an encrypted channel only. Doesn't this protect against these kind of manipulation attacks?", one of them asks.

---

**State** whether DNS-over-TLS as stated above prevents DNS cache poisioning attacks. **Justify** your answer. 

YOUR ANSWER HERE

##### b) (2 points)

The next customer has recently implemented DNSSec on his site. "We have just enabled DNSSec for our domain (`customer.exercise`). This means that our stub resolver is now secure, and our employees (that use the stub) are immune against such attacks, no matter which site they visit, right?".

---

**State and justify** whether the employees of the second customer are secured against cache poisioning attacks. Differentiate between the employees visiting their companies website (`customer.exercise`) and arbitrary other websites.

YOUR ANSWER HERE

##### c) (3 points)

The first customer now asks again. He has not heard of DNSSec before and is pretty confused.

---

**Briefly explain** how DNSSec work. **Name** all keys that occur and **describe** their function.

YOUR ANSWER HERE

### Task 1.3 Bayesian Bullocks (3 points)

It seems like you were pretty confusing, and the customers definetly are more interested in the SecureStub solution now. One of it's big marketing claim is that it can reliably detect cache poisioning even on domains that do not have DNSSec enabled.
However, your customers are worried about the reliablity of that detection mechansim. The sales people are quick to point out some numbers: "Our advanced algorithms use state-of-the-art AI methods. An attack is detected with 99,999% probability."

This sounds impressing, but the customers are also worried about the false alarm rate, to which the sales people respond that there is only a one-in-a-million chance that a benign response is classified as an attack. You notice that this is not actually the false alarm rate, and start calculating the actual false alarm rate under the assumption that only 0.00001% of the total traffic is actually malicious.

---

Use the [Bayes theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem#Statement_of_theorem) to **calculate** the false alarm rate ($P(\neg attack | alarm))$, given that $P(alarm | attack) = 0.99999, P(alarm | \neg attack) = 0.0000001$, and $P(attack) = 0.0000001$. **Show** your calculations.

YOUR ANSWER HERE

## 2. E-Mail security (4.5 points)

With the successful sale of the SecureStub DNS stub server, the company starts to think about expanding the security products portfolio to also cover E-Mails. Since the company already has expertiece in "state-of-the-art AI methodologies", they quickly are able to build a decent spam filter. But at some point, it is noticed that E-Mails usually are not encrypted at all. At that point, the product owner turns to you.

### Task 2.1: PGP - Web of Trust (1.5 points)

Being more competent that your previous managers, the product owner already has done some research on ways to encrypt E-Mails, and discovered PGP and the web of trust. To better understand how it works, he came up with the following example network.

---

![](./pgp.png)

In this example, Alice assigns a weight of $\frac{1}{2}$ to fully trusted introducers and a weight of $\frac{1}{3}$ to partially trusted introducers. Alice trusts herself with a weight of 1. **Calculate** the key legitimacies Alice assigns to each keys, and enter them in the cell below.

In [ ]:
def return_key_legitimacies() -> dict[str, int]:
    return {
        "Alice": 1, # Already did that for you :)
        "A": 0, # You will to do the rest on your own :)
        "B": 0,
        "C": 0,
        "D": 0,
        "E": 0
    }

def just_ignore_this_function() -> dict[str, int]:
    # Please just ignore this function.
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 2.2: TLS vs. PGP and S/MIME (2 point)

That helped the product owner a lot with understanding how the web of trust works. However, that is just a mechanism to establish the authenticity of public keys, and not how the keys are used. When it comes to actual protocols using encryption, he notes that traditional E-Mail protocols such as SMTP and IMAP can be used alongside TLS, which is also known as SMTPs or IMAPs. But there are also other protocols such as PGP or S/MIME.

---

**Explain** the fundamental difference between the usage of TLS (e.g. with SMTPs or IMAPs) and the usage of PGP or S/MIME during e-mail transfer. Explicitly name who has access to the messages and who is able to modify them.

YOUR ANSWER HERE

### Task 2.3: PGP vs S/MIME (1 point)

The product owner again thanks you for your valuable insights. "It seems like we should push PGP or S/MIME then. We have already discussed how PGP establishes key authenticity using the web of trust, but how does S/MIME know which public keys should be trusted?"

---

**Briefly explain** how the trustworthiness of a S/MIME certificate is determined.

YOUR ANSWER HERE

## 3. Feedback (0.5 points)

You made it through another assignment! Since we want to know how it went and how we might improve the exercises, we include the following task. Here, you can write constructive feedback! You even get 0.5 points for it if you write anything. But don't worry, we do not grade the content itself!

YOUR ANSWER HERE